In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_predict

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re


from scipy.spatial import cKDTree

In [2]:
m=pd.read_csv('plant_pest_dataset.csv')
m.head(30)

,id,observed_on,latitude,longitude,phenophase,scientific_name,common_name,taxon_id,doy,biome,T2M,PRECTOTCORR,SWGDN,biome_cat,scientific_name_pest,common_name_pest
0,310657266,2025-08-31,40.799434,-111.012697,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,292.79025,0.000011,288.58110,H,Empoasca fabae,Potato Leafhopper
1,310812803,2025-08-31,40.690640,-110.903167,Flowering,Achillea millefolium,common yarrow,52821,243,27.0,290.41034,0.000012,279.16245,H,Ostrinia nubilalis,European Corn Borer Moth
2,310648485,2025-08-31,43.939639,-87.719908,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,294.50253,0.000030,255.68756,H,Popillia japonica,Japanese Beetle
3,310647240,2025-08-31,44.741903,-65.519220,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,290.05582,0.000014,265.07320,H,Lymantria dispar,Spongy Moth
4,310482483,2025-08-31,42.146311,-77.131258,Flowering,Achillea millefolium,common yarrow,52821,243,25.0,293.99230,0.000021,266.84604,H,Leptinotarsa decemlineata,Colorado Potato Beetle
5,310457618,2025-08-31,58.544962,31.377577,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,288.90690,0.000045,192.62874,H,Leptinotarsa decemlineata,Colorado Potato Beetle
6,310215347,2025-08-30,63.558598,26.662874,Flowering,Achillea millefolium,common yarrow,52821,242,27.0,287.83690,0.000021,182.90506,H,Plutella xylostella,Diamondback Moth
7,310201801,2025-08-30,60.234195,24.946592,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,289.69420,0.000028,196.11778,H,Plutella xylostella,Diamondback Moth
8,311993972,2025-08-30,42.558192,2.112421,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,290.25610,0.000025,251.79330,H,Lymantria dispar,Spongy Moth
9,311993941,2025-08-30,42.419870,2.010164,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,290.25610,0.000025,251.79330,H,Lymantria dispar,Spongy Moth


In [3]:
KOPPEN_CLASSES = {
    1: "Af",  2: "Am",  3: "Aw",
    4: "BWh", 5: "BWk", 6: "BSh", 7: "BSk",
    8: "Csa", 9: "Csb", 10: "Cwa", 11: "Cwb", 12: "Cfa", 13: "Cfb", 14: "Cfc",
    15: "Dsa", 16: "Dsb", 17: "Dsc", 18: "Dsd",
    19: "Dwa", 20: "Dwb", 21: "Dwc", 22: "Dwd",
    23: "Dfa", 24: "Dfb", 25: "Dfc", 26: "Dfd",
    27: "ET",  28: "EF"
}

# 🧠 Map biome number to Köppen code
m['biome_cat'] = m['biome'].map(KOPPEN_CLASSES).fillna("Unknown")


In [4]:
m.head(30)

,id,observed_on,latitude,longitude,phenophase,scientific_name,common_name,taxon_id,doy,biome,T2M,PRECTOTCORR,SWGDN,biome_cat,scientific_name_pest,common_name_pest
0,310657266,2025-08-31,40.799434,-111.012697,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,292.79025,0.000011,288.58110,Dfd,Empoasca fabae,Potato Leafhopper
1,310812803,2025-08-31,40.690640,-110.903167,Flowering,Achillea millefolium,common yarrow,52821,243,27.0,290.41034,0.000012,279.16245,ET,Ostrinia nubilalis,European Corn Borer Moth
2,310648485,2025-08-31,43.939639,-87.719908,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,294.50253,0.000030,255.68756,Dfd,Popillia japonica,Japanese Beetle
3,310647240,2025-08-31,44.741903,-65.519220,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,290.05582,0.000014,265.07320,Dfd,Lymantria dispar,Spongy Moth
4,310482483,2025-08-31,42.146311,-77.131258,Flowering,Achillea millefolium,common yarrow,52821,243,25.0,293.99230,0.000021,266.84604,Dfc,Leptinotarsa decemlineata,Colorado Potato Beetle
5,310457618,2025-08-31,58.544962,31.377577,Flowering,Achillea millefolium,common yarrow,52821,243,26.0,288.90690,0.000045,192.62874,Dfd,Leptinotarsa decemlineata,Colorado Potato Beetle
6,310215347,2025-08-30,63.558598,26.662874,Flowering,Achillea millefolium,common yarrow,52821,242,27.0,287.83690,0.000021,182.90506,ET,Plutella xylostella,Diamondback Moth
7,310201801,2025-08-30,60.234195,24.946592,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,289.69420,0.000028,196.11778,Dfd,Plutella xylostella,Diamondback Moth
8,311993972,2025-08-30,42.558192,2.112421,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,290.25610,0.000025,251.79330,Dfd,Lymantria dispar,Spongy Moth
9,311993941,2025-08-30,42.419870,2.010164,Flowering,Achillea millefolium,common yarrow,52821,242,26.0,290.25610,0.000025,251.79330,Dfd,Lymantria dispar,Spongy Moth


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def engineer_features(df):
    """
    Applies feature engineering to the plant and pest dataset, including
    advanced handling for precipitation data.

    Args:
        df (pd.DataFrame): The raw input DataFrame.

    Returns:
        pd.DataFrame: The DataFrame with new, engineered features.
    """
    # Make a copy to avoid changing the original DataFrame
    df_processed = df.copy()

    # --- 1. Sort data by date (CRUCIAL for rolling calculations) ---
    # Ensure 'observed_on' is a datetime object before sorting
    df_processed['observed_on'] = pd.to_datetime(df_processed['observed_on'])
    df_processed = df_processed.sort_values(by='observed_on').reset_index(drop=True)

    # --- 2. Engineering Time-Based Features ---
    df_processed['doy_sin'] = np.sin(2 * np.pi * df_processed['doy'] / 365.25)
    df_processed['doy_cos'] = np.cos(2 * np.pi * df_processed['doy'] / 365.25)
    
    # --- 3. Engineering Advanced Precipitation Features ---
    # Captures recent moisture trend (total rain in last 7 days)
    df_processed['precip_sum_7d'] = df_processed['PRECTOTCORR'].rolling(window=7, min_periods=1).sum()
    
    # Directly models drought conditions (days since it last rained)
    s = df_processed['PRECTOTCORR'].ne(0).cumsum()
    df_processed['days_since_rain'] = df_processed.groupby(s).cumcount()

    # Log transform to manage the wide range of rainfall values
    df_processed['precip_log'] = np.log1p(df_processed['PRECTOTCORR'])

    # Simple binary feature for whether a rain event occurred
    df_processed['did_rain'] = (df_processed['PRECTOTCORR'] > 0).astype(int)

    # --- 4. Engineering Climate and Weather Interaction Features ---
    df_processed['temp_x_radiation'] = df_processed['T2M'] * df_processed['SWGDN']
    df_processed['drought_index'] = df_processed['T2M'] / (df_processed['PRECTOTCORR'] + 1)

    # --- 5. Feature Scaling ---
    # Update the list to include all new numerical features
    numerical_features = [
        'latitude', 'longitude', 'T2M', 'PRECTOTCORR', 'SWGDN',
        'doy_sin', 'doy_cos', 'precip_sum_7d', 'days_since_rain', 'precip_log',
        'temp_x_radiation', 'drought_index'
    ]
    
    features_to_scale = [f for f in numerical_features if f in df_processed.columns]
    
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_processed[features_to_scale])
    df_scaled = pd.DataFrame(scaled_data, columns=features_to_scale, index=df_processed.index)
    
    # Drop original numerical columns and replace with scaled versions
    df_final = df_processed.drop(columns=features_to_scale)
    df_final = pd.concat([df_final, df_scaled], axis=1)

    return df_final

# --- How to use the function ---

# 1. Assume your DataFrame 'm' is already loaded
# For example:
# m = pd.read_csv('your_data.csv') 

# 2. Apply the feature engineering function to your DataFrame 'm'
# Ensure your DataFrame 'm' has the required columns: 
# 'observed_on', 'doy', 'T2M', 'SWGDN', 'PRECTOTCORR', 'latitude', 'longitude'
m_engineered = engineer_features(m)

# 3. Display the result
print("--- Data After Advanced Feature Engineering ---")
print("New features for precipitation like 'precip_sum_7d' and 'days_since_rain' have been added.")
print("All numerical features have been engineered and scaled.\n")

print(m_engineered.head())

--- Data After Advanced Feature Engineering ---
New features for precipitation like 'precip_sum_7d' and 'days_since_rain' have been added.
All numerical features have been engineered and scaled.

         id observed_on phenophase              scientific_name  \
0   2039263  2015-09-30   Fruiting            Fagus grandifolia   
1   2078741  2015-10-02   Fruiting            Fagus grandifolia   
2   2084970  2015-10-05   Fruiting  Parthenocissus quinquefolia   
3   2333801  2015-10-14   Fruiting  Parthenocissus quinquefolia   
4  66321481  2015-11-27   Fruiting           Carnegiea gigantea   

        common_name  taxon_id  doy  biome biome_cat      scientific_name_pest  \
0    American beech     49202  273   26.0       Dfd      Diabrotica virgifera   
1    American beech     49202  275   26.0       Dfd      Diabrotica virgifera   
2  Virginia creeper     50278  278   26.0       Dfd      Diabrotica virgifera   
3  Virginia creeper     50278  287   14.0       Cfc  Cnaphalocrocis medinalis

In [2]:
# ==============================================================================
#                      ALL-IN-ONE SCRIPT
# ==============================================================================
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
import joblib
import warnings

# Suppress unnecessary warnings for a cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

def engineer_features(df):
    """Applies feature engineering to the raw input data."""
    if 'observed_on' not in df.columns:
        df['observed_on'] = pd.to_datetime(df['doy'], unit='D', origin='2023-01-01')
    
    df['doy_sin'] = np.sin(2 * np.pi * df['doy'] / 365.25)
    df['doy_cos'] = np.cos(2 * np.pi * df['doy'] / 365.25)
    df['temp_x_radiation'] = df['T2M'] * df['SWGDN']
    df['drought_index'] = df['T2M'] / (df['PRECTOTCORR'] + 1)
    return df

def get_final_scenarios(user_input_dict):
    """
    Takes 5 user inputs, loads the trained model, and returns a list of 10 scenarios.
    """
    print("\nLoading model and lookup tables for prediction...")
    try:
        # Load all the files created by the training script
        model = joblib.load('combined_model.joblib')
        encoder = joblib.load('combined_encoder.joblib')
        training_columns = joblib.load('training_columns.joblib')
        with open('scenario_to_plants_lookup.json', 'r') as f:
            scenario_to_plants_lookup = json.load(f)
        with open('name_lookups.json', 'r') as f:
            name_lookups = json.load(f)
    except FileNotFoundError:
        return "[ERROR] Model files not found. Please run the training part of the script first."

    # Prepare your custom input for the model
    user_df = pd.DataFrame([user_input_dict])
    user_df_engineered = pd.get_dummies(user_df, columns=['biome_cat'])
    user_df_engineered = engineer_features(user_df_engineered)
    X_user = user_df_engineered.reindex(columns=training_columns, fill_value=0)

    # Get the model's prediction
    prediction_encoded = model.predict(X_user)
    most_likely_event = encoder.inverse_transform(prediction_encoded)[0]
    print(f"Most Likely Event Predicted: '{most_likely_event}'")

    # Look up the associated plants for the predicted event
    top_10_plants = scenario_to_plants_lookup.get(most_likely_event, [])
    if not top_10_plants:
        return f"[INFO] The predicted event '{most_likely_event}' had no associated plants in the training data."

    # Build the final list of 10 scenarios
    phenophase, pest_sci_name = most_likely_event.split(' | ')
    final_scenarios = []
    for plant_sci_name in top_10_plants:
        scenario = {
            "scientific_name": plant_sci_name,
            "common_name": name_lookups['plant_names'].get(plant_sci_name, "N/A"),
            "phenophase": phenophase,
            "scientific_name_pest": pest_sci_name,
            "common_name_pest": name_lookups['pest_names'].get(pest_sci_name, "N/A")
        }
        final_scenarios.append(scenario)
        
    return final_scenarios

# ==============================================================================
# MAIN SCRIPT EXECUTION (TRAINING & PREDICTION)
# ==============================================================================
if __name__ == "__main__":
    training_successful = False
    try:
        # --- PART 1: TRAINING ---
        print("--- Starting Model Training Workflow ---")
        
        m = pd.read_csv('plant_pest_dataset.csv')
        
        KOPPEN_CLASSES = {
            1: "Af",  2: "Am",  3: "Aw", 4: "BWh", 5: "BWk", 6: "BSh", 7: "BSk",
            8: "Csa", 9: "Csb", 10: "Cwa", 11: "Cwb", 12: "Cfa", 13: "Cfb", 14: "Cfc",
            15: "Dsa", 16: "Dsb", 17: "Dsc", 18: "Dsd", 19: "Dwa", 20: "Dwb", 
            21: "Dwc", 22: "Dwd", 23: "Dfa", 24: "Dfb", 25: "Dfc", 26: "Dfd",
            27: "ET",  28: "EF"
        }
        m['biome_cat'] = m['biome'].map(KOPPEN_CLASSES).fillna("Unknown")

        m['combined_target'] = m['phenophase'] + ' | ' + m['scientific_name_pest']
        
        print("Creating lookup tables...")
        scenario_to_plants_lookup = m.groupby('combined_target')['scientific_name'].apply(
            lambda x: x.value_counts().head(10).index.tolist()
        ).to_dict()
        plant_names = m.drop_duplicates('scientific_name').set_index('scientific_name')['common_name'].to_dict()
        pest_names = m.drop_duplicates('scientific_name_pest').set_index('scientific_name_pest')['common_name_pest'].to_dict()
        name_lookups = {'plant_names': plant_names, 'pest_names': pest_names}

        m_engineered = pd.get_dummies(m, columns=['biome_cat'])
        m_engineered = engineer_features(m_engineered)
        
        y = m_engineered['combined_target']
        feature_cols_to_drop = [
            'id', 'observed_on', 'taxon_id', 'phenophase', 'scientific_name_pest',
            'scientific_name', 'common_name', 'common_name_pest', 'doy', 'biome', 'combined_target'
        ]
        X = m_engineered.drop(columns=feature_cols_to_drop, errors='ignore')
        training_columns = X.columns.tolist()

        combined_encoder = LabelEncoder()
        y_encoded = combined_encoder.fit_transform(y)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)
        
        print("Training single model for combined target...")
        model_gpu = xgb.XGBClassifier(device='cuda', objective='multi:softprob', eval_metric='mlogloss', use_label_encoder=False, random_state=42)
        model_gpu.fit(X_train, y_train)
        print("Model training complete.\n")
        
        joblib.dump(model_gpu, 'combined_model.joblib')
        joblib.dump(combined_encoder, 'combined_encoder.joblib')
        joblib.dump(training_columns, 'training_columns.joblib')
        with open('scenario_to_plants_lookup.json', 'w') as f: json.dump(scenario_to_plants_lookup, f)
        with open('name_lookups.json', 'w') as f: json.dump(name_lookups, f)
        print("Model and lookup tables saved successfully.\n")

        training_successful = True

    except FileNotFoundError:
        print("\n[ERROR] 'plant_pest_dataset.csv' not found. Please check the file name and location.")
    except Exception as e:
        print(f"\n[ERROR] An error occurred during training: {e}")

    

--- Starting Model Training Workflow ---
Creating lookup tables...
Training single model for combined target...
Model training complete.

Model and lookup tables saved successfully.



In [3]:
my_input = {
    "biome_cat": "Bwh",     # Climate code (e.g., "Dfc", "ET", "BWh")
    "doy": 27,             # Day of the year (1-365)
    "T2M": 290.0,           # Temperature in Kelvin (e.g., 290K is ~17°C)
    "PRECTOTCORR": 10.5,    # Total precipitation in mm
    "SWGDN": 150.0          # Sunlight intensity
}
# =======================================================

# Call the function with your custom inputs
final_output = get_final_scenarios(my_input)

# Print the final result
print("\n✅ FINAL PREDICTED SCENARIOS:")
print(json.dumps(final_output, indent=2))


Loading model and lookup tables for prediction...
Most Likely Event Predicted: 'Flowering | Maruca vitrata'

✅ FINAL PREDICTED SCENARIOS:
[
  {
    "scientific_name": "Solidago altissima",
    "common_name": "tall goldenrod",
    "phenophase": "Flowering",
    "scientific_name_pest": "Maruca vitrata",
    "common_name_pest": "Mung Bean Moth"
  },
  {
    "scientific_name": "Mimosa pudica",
    "common_name": "Sensitive Plant",
    "phenophase": "Flowering",
    "scientific_name_pest": "Maruca vitrata",
    "common_name_pest": "Mung Bean Moth"
  },
  {
    "scientific_name": "Asystasia intrusa",
    "common_name": "African Coromandel",
    "phenophase": "Flowering",
    "scientific_name_pest": "Maruca vitrata",
    "common_name_pest": "Mung Bean Moth"
  },
  {
    "scientific_name": "Catharanthus roseus",
    "common_name": "Madagascar Periwinkle",
    "phenophase": "Flowering",
    "scientific_name_pest": "Maruca vitrata",
    "common_name_pest": "Mung Bean Moth"
  },
  {
    "scienti

In [ ]:
# =======================================================
#               MODEL EVALUATION SCRIPT
# =======================================================
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json
import joblib
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Suppress unnecessary warnings
warnings.filterwarnings('ignore', category=UserWarning)

def engineer_features(df):
    """Applies feature engineering to the raw input data."""
    if 'observed_on' not in df.columns:
        df['observed_on'] = pd.to_datetime(df['doy'], unit='D', origin='2023-01-01')
    
    df['doy_sin'] = np.sin(2 * np.pi * df['doy'] / 365.25)
    df['doy_cos'] = np.cos(2 * np.pi * df['doy'] / 365.25)
    df['temp_x_radiation'] = df['T2M'] * df['SWGDN']
    df['drought_index'] = df['T2M'] / (df['PRECTOTCORR'] + 1)
    return df

if __name__ == "__main__":
    try:
        # --- Step 1: Load the trained model and encoder ---
        print("Loading trained model and helper files...")
        model = joblib.load('combined_model.joblib')
        encoder = joblib.load('combined_encoder.joblib')

        # --- Step 2: Recreate the exact same test set ---
        # To evaluate correctly, we must process the original data in the same way.
        print("Loading and preprocessing original dataset to get the test set...")
        m = pd.read_csv('plant_pest_dataset.csv')
        
        KOPPEN_CLASSES = {
            1: "Af",  2: "Am",  3: "Aw", 4: "BWh", 5: "BWk", 6: "BSh", 7: "BSk",
            8: "Csa", 9: "Csb", 10: "Cwa", 11: "Cwb", 12: "Cfa", 13: "Cfb", 14: "Cfc",
            15: "Dsa", 16: "Dsb", 17: "Dsc", 18: "Dsd", 19: "Dwa", 20: "Dwb", 
            21: "Dwc", 22: "Dwd", 23: "Dfa", 24: "Dfb", 25: "Dfc", 26: "Dfd",
            27: "ET",  28: "EF"
        }
        m['biome_cat'] = m['biome'].map(KOPPEN_CLASSES).fillna("Unknown")
        m['combined_target'] = m['phenophase'] + ' | ' + m['scientific_name_pest']
        
        m_engineered = pd.get_dummies(m, columns=['biome_cat'])
        m_engineered = engineer_features(m_engineered)
        
        y = m_engineered['combined_target']
        feature_cols_to_drop = [
            'id', 'observed_on', 'taxon_id', 'phenophase', 'scientific_name_pest',
            'scientific_name', 'common_name', 'common_name_pest', 'doy', 'biome', 'combined_target'
        ]
        X = m_engineered.drop(columns=feature_cols_to_drop, errors='ignore')

        y_encoded = encoder.transform(y)
        
        # Use the same train_test_split parameters to get the identical test set
        X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)
        
        # --- Step 3: Evaluate the model ---
        print("\n--- Evaluating Model Performance on Test Set ---")
        
        # Make predictions on the unseen test data
        y_pred_encoded = model.predict(X_test)
        
        # Calculate and print the overall accuracy
        accuracy = accuracy_score(y_test, y_pred_encoded)
        print(f"Overall Accuracy: {accuracy:.4f}\n")
        
        # For a more detailed report, decode the labels back to text
        y_test_decoded = encoder.inverse_transform(y_test)
        y_pred_decoded = encoder.inverse_transform(y_pred_encoded)
        
        # Print the detailed classification report
        print("Classification Report:")
        print(classification_report(y_test_decoded, y_pred_decoded, zero_division=0))

    except FileNotFoundError:
        print("\n[ERROR] Could not find the dataset or saved model files.")
        print("Please make sure 'plant_pest_dataset.csv', 'combined_model.joblib', and 'combined_encoder.joblib' are in the same folder as this script.")
    except Exception as e:
        print(f"\n[ERROR] An error occurred during evaluation: {e}")

Loading trained model and helper files...
Loading and preprocessing original dataset to get the test set...

--- Evaluating Model Performance on Test Set ---
Overall Accuracy: 0.5498

Classification Report:
                                                precision    recall  f1-score   support

                    Budding | Aedes albopictus       0.12      0.01      0.02       235
                 Budding | Agrilus planipennis       0.30      0.06      0.11      1354
                     Budding | Agrotis ipsilon       0.23      0.04      0.07       149
                 Budding | Bactrocera dorsalis       0.12      0.03      0.04        38
               Budding | Brevicoryne brassicae       0.28      0.06      0.11       726
                  Budding | Ceratitis capitata       0.38      0.11      0.16       352
            Budding | Cnaphalocrocis medinalis       0.27      0.05      0.08        88
                     Budding | Cydia pomonella       0.33      0.06      0.10       437
